<a href="https://colab.research.google.com/github/wonderfulvamsi/Nerual-Style-Transfer-Web-App/blob/master/NSTapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting all the dependencies 
===

In [1]:
%%writefile NSTapp.py


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image

import torchvision.transforms as transforms
import torchvision.models as models

import copy

import streamlit as st

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim





#--------------------------------------FrontEnd
st.title('Neural Style Transfer <3')
st.write('This AI lets you to impose the look of a pic (style image) on to another pic(content image)')
st.markdown('set style coeff to 0.2 & content coeff to 0.8 with more no of iterations for achiving decent results')

content_file = st.file_uploader("Upload Content image...", type="jpg")

if content_file is not None:
    st.image(content_file, caption='Uploaded Content Image.', use_column_width=True)

    
style_file = st.file_uploader("Upload Style image...", type="jpg")

if style_file is not None:
    st.image(style_file, caption='Uploaded Style Image.', use_column_width=True)


epochz=150

sxx=1
cxx=1

epochz = st.slider('Number of iterations',min_value=100,max_value=1000,step=50)
sxx = st.slider('Style coeff',min_value=0.1,max_value=2.0,step=0.1)
cxx = st.slider('Content coeff',min_value=0.1,max_value=2.0,step=0.1)

#--------------------------------------BackEnd+++++interactive variables

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
st.write('Running on..',device,'...')

# desired size of the output image
imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu

loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor



def image_loader(image_name):
    image = image_name
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

#-----------------------------+++++Take input images from FrontEnd

if style_file and content_file:
  s = Image.open(style_file)
  c = Image.open(content_file)
  s = s.resize(c.size)
  style_img = image_loader(s)
  content_img = image_loader(c)
  if style_img.size() != content_img.size():
    st.write("we need to import style and content images of the same size")
  input_img = content_img.clone()

#---------------------------------++++++Show the uploded images

unloader = transforms.ToPILImage()  # reconvert into PIL image

def imshow(tensor, title):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    st.image(image,title)

#--------------------------------Loss functions

class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input
    
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

#---------------------------------Pre-trained model & Normalization
cnn = models.vgg19(pretrained=True).features.to(device).eval()

cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

#------------------------------Calculate losses

# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

#-------------------------------------NST++++++


def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps= epochz,
                       style_weight=1000000, content_weight=1):

    st.write('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)
    optimizer = get_input_optimizer(input_img)

    st.write('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = sxx*style_score + cxx*content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                st.write("run {}:".format(run))
                st.write('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                st.write()

            return sxx*style_score + cxx*content_score

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

#-------------------------------Run NST++++++++++++show result 

if st.button("Submit"):
  if style_file and content_file:
    input_img = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                   content_img, style_img, input_img)
    imshow(input_img, title='Resultant Image')
  else:
    st.write("First you gotta upload both the images..!!")

Overwriting NSTapp.py


In [2]:

!pip install streamlit -q

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

--2020-07-02 20:44:27--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.220.228.61, 52.2.226.158, 3.223.249.113, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.220.228.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.22’

ngrok-stable-linux- 100%[===================>]  13.13M  11.7MB/s    in 1.1s    

2020-07-02 20:44:29 (11.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.22’ saved [13773305/13773305]

replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://a4e49172174f.ngrok.io


In [ ]:
!streamlit run NSTapp.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.204.228.113:8501

/content/NSTapp.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = torch.tensor(mean).view(-1, 1, 1)
/content/NSTapp.py:149: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = torch.tensor(std).view(-1, 1, 1)


In [ ]:
!pip install pipreqs